## Data Analysis for Physics and Astronomy
## Sommersemster 2025 
# Assignment 2

Due: **10:00 27. May 2025**

Discussion: **12:00 27. May 2025**

**Online submission** at via github classroom  

# Import and understand scientific data [30 Points]

Describe each of the following file types, then load and visualize the corresponding file's contents.

In [1]:
%matplotlib inline

**a.** FITS: Open and visualise the data in `Data/CII_O1.fits`. **10 points**

The important package is from the astropy libary: [DOCUMENTATION](https://docs.astropy.org/en/stable/io/fits/)

Every Header Data Unit (HDU) normally has two components: header and data. In astropy these two components are accessed through the two attributes of the HDU, `hdu.header` and `hdu.data`.

What is important to know is that the data contained in the HDU has the shape of the givin axes, but it is not specifically those values. For this FITS file, we have Intensity values for axes of 'Frequency', 'RA', 'DEC', and 'Stokes'. The units of the intensity data is given by `BUNIT`: Kelvin. This is typical for radio observational data. The axes can be rebuilt using `CRVAL`, `CRPIX`  and `CDELT` for each axis, and the axes must be linearly-spaced. The units of these axes must be inferred in this case, but typically comments are left for anything that is abnormal. You can find a description of the heder labels [here](https://diffractionlimited.com/help/maximdl/FITS_File_Header_Definitions.htm).

Since axes 2, 3, and 4 have length 1, we can simplify our data to two dimensions: frequency and intensity. One super confusing thing about extracting the data (at least I find it super confusing) is that the axes in the extracted numpy array are reversed compared to the header. This is due to the difference between how axezs are defined in numpy and in the HDU standard. So to access the first axis in the data you would have to use something like `hdul[0].data[0,0,0,:]`.

Now you just need to plot the data and you are done.

First print the header info.

In [3]:
from astropy.io import fits
import pprint

# add your code here

Then access the spectrum data. The `.data` attribute of a table HDU contains the data of the table.

In [ ]:
# add your code here

The x axis is frequency as written in the header. Calculate the frequency axis from the header value by using the formula $(i-(crpix1-1))*cdelt1+crval1$

In [ ]:
# add your code here

Plot the spectrum. Use the frequency values as x-value. Determine the correct Units for the y-axis.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

# here your code

b. HDF4: Open and visualise the data in 2006001-2006005.s0454pfrt-bsst.hdf. 10 points

You can use pyhdf to obtain extract the HDF4 data. This particular file contains a dictionary with three data objects: one for the two axes, and another containing the data. The axes thus do not need to be rebuilt as with the FITS standard.

In [ ]:
from pyhdf.SD import *
import matplotlib.pyplot as plt
import pprint

file = SD('Data/2006001-2006005.s0454pfrt-bsst.hdf', SDC.READ)

dataset = file.datasets()

# here your code

**c.** HDF5: Open and visualise the data in `3B-HHR.MS.MRG.3IMERG.20141001-S090000-E092959.0540.V05B.HDF5`. **10 points**

It is worth noting that `pandas` also contains HDF5 functionality, but it is specifically for `pandas` dataframes. It cannot read standard HDF5 files. We instead have to use `h5py`.

In [ ]:
import h5py

hdffilename = "Data/3B-HHR.MS.MRG.3IMERG.20141001-S090000-E092959.0540.V05B.HDF5"
hdf_file = h5py.File(hdffilename,'r')

To see the top-level meta data,

In [ ]:
topmeta=hdf_file['/']
print(topmeta.attrs.keys())

In [ ]:
fileheader = topmeta.attrs.__getitem__('FileHeader')
print(fileheader.decode('UTF-8'))

In [ ]:
fileinfo = topmeta.attrs.__getitem__('FileInfo')
print(fileinfo.decode('UTF-8'))

How can we get an overview of the structure?

In [ ]:
print(list(hdf_file.keys()))

But this does not show the whole tree structures.

In [ ]:
def PrintDataset(name, obj):
    if isinstance(obj,h5py.Dataset):
        print(name)
        print('\t',obj)
hdf_file.visititems(PrintDataset)

There is a bunch of different data contained in this file. We will examine one of these - 'HQobservationTime' - in more detail.

In [ ]:
meta=hdf_file['Grid/lon']
print(meta.attrs.keys())

fileinfo = meta.attrs.__getitem__('Units')
print(fileinfo.decode('UTF-8'))

meta=hdf_file['Grid/lat']
print(meta.attrs.keys())

fileinfo = meta.attrs.__getitem__('Units')
print(fileinfo.decode('UTF-8'))

meta=hdf_file['Grid/HQobservationTime']
print(meta.attrs.keys())

fileinfo = meta.attrs.__getitem__('DimensionNames')
print(fileinfo.decode('UTF-8'))
fileinfo = meta.attrs.__getitem__('units')
print(fileinfo.decode('UTF-8'))
fileinfo = meta.attrs.__getitem__('Units')
print(fileinfo.decode('UTF-8'))

In [ ]:
hdf_file.close()

It would be useful if we can store the contents into pandas dataframe.

In [ ]:
import pandas as pd
import numpy as np

def get_df_from_hdf(hdffilename,datasets):
    header = {'filename':hdffilename}

    with h5py.File(hdffilename,'r') as f:
        for colname, dataset in datasets.items():
            ds = f[dataset]
            header[colname] = [ds[()]]

    df = pd.DataFrame(header)
    return df

datasets = {'lat':'Grid/lat', 'lon':'Grid/lon', 'HQobservationTime':'Grid/HQobservationTime'}
df = get_df_from_hdf(hdffilename,datasets)
print(df)

Visualize the data 'HQobservationTime' as a function of the latitute 'lat' and longitude 'lon'.

In [ ]:
# your code goes here

# 2. Averaging spectral radio data (DATA EXERCISE) [30 Points]

In this exercise you will work on a data set of artificial radio observation `radio-map.fits`. It is a map of 5x5
spectra with 201 measured frequency/velocity channels (channel width is unity). If you number all spectra
from 1 to 25 we use the following scheme to assign spectra to positions on the map (spectrum 1 is the top
left spectrum):

In [6]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import pprint

In [ ]:
m = fits.open("Data/radio-map.fits")
pprint.pprint(m[0].header)
spectra = m[0].data

**a.** Compute the total “integrated-intensity” map of the observations, i.e. integrate the spectra over the full spectral range for all positions and plot the 5x5 map in a suitable way. **5 Points**

In [ ]:
# enter your code

**b.** Compute two channel maps by integrating over the frequency channels 50-100 and 100-150. Compare the two maps. **5 Points**

In [ ]:
# enter your code

**c.** Compute the average spectrum, by averaging all 25 positions. **5 Points**

In [ ]:
# enter your code

**d.** Plot every spectrum and overlay the average spectrum. Describe how the emission changes across the map. In particular how the line center position, the peak height and the line widths behave. **15 Points**

In [ ]:
# enter your code